In [7]:
import os
import json

result_dir = "../exp_results"
num_cores = [1]
num_tasks = [3,6,9,12]
utilizations = [0.4, 0.6, 0.8]
num_tasksets = 30
summary_path = result_dir + "/summary.csv"

# if summary file exists, remove it
if os.path.exists(summary_path):
    os.remove(summary_path)
    
header = "numCores,numTasks,utilization,tasksetIndex,realLinux_schedulability"
with open(summary_path, "a") as f:
    f.write(header + "\n")

for num_core in num_cores:
    for num_task in num_tasks:
        for utilization in utilizations:
            for i in range(num_tasksets):
                result_file = result_dir + "/details/" + str(num_core) + "cores/" + str(num_task) + "tasks/" + str(utilization) + "utilization/result_" + str(i) + ".json"
                with open(result_file, 'r') as file:
                    tasks_result = json.load(file)
                assert len(tasks_result) == num_task
                
                realLinux_schedulability = True
                for task_result in tasks_result:
                    if task_result["wcrt_ns"] > task_result["deadline_ns"]:
                        realLinux_schedulability = False
                        break
                with open(summary_path, "a") as f:
                    f.write(str(num_core) + "," + str(num_task) + "," + str(utilization) + "," + str(i) + "," + str(realLinux_schedulability) + "\n")
                


In [4]:
results_df.columns

Index(['task_name', 'core_index', 'nice_value', 'deadline_ns', 'wcrt_ns',
       'wcet_ns', 'avg_response_time_ns', 'response_time_ns'],
      dtype='object')

In [15]:
import json
import pandas as pd

# analyze the exp with non_rt_task
result_dir = "../exp_results_non_RT"
schedulers = ["CFS", "FIFO", "RR", "RM"]

# set task_info in dataset
result_path = result_dir + "/" + schedulers[-1] + "_result.json"
results_df = pd.read_json(result_path)
results_df = results_df[["task_name", "deadline_ns", "wcet_ns", "priority"]]

for scheduler in schedulers:
    result_path = result_dir + "/" + scheduler + "_result.json"
    result_df = pd.read_json(result_path)
    result_df = result_df[["task_name", "wcrt_ns"]]
    result_df.columns = ["task_name", scheduler + "_wcrt_ns"]
    results_df = pd.merge(results_df, result_df, on="task_name")

# convert to ms
results_df["deadline_ms"] = results_df["deadline_ns"] / 1000000
results_df["wcet_ms"] = results_df["wcet_ns"] / 1000000
results_df["CFS_wcrt_ms"] = results_df["CFS_wcrt_ns"] / 1000000
results_df["FIFO_wcrt_ms"] = results_df["FIFO_wcrt_ns"] / 1000000
results_df["RR_wcrt_ms"] = results_df["RR_wcrt_ns"] / 1000000
results_df["RM_wcrt_ms"] = results_df["RM_wcrt_ns"] / 1000000

# round to 2 decimal places
results_df = results_df.round(2)

# drop ns columns
results_df = results_df.drop(["deadline_ns", "wcet_ns", "CFS_wcrt_ns", "FIFO_wcrt_ns", "RR_wcrt_ns", "RM_wcrt_ns"], axis=1)
results_df

,task_name,deadline_ms,wcet_ms,CFS_wcrt_ms,FIFO_wcrt_ms,RR_wcrt_ms,RM_wcrt_ms
0,task1,100.0,2.79,14.53,855.33,336.32,6.12
1,task2,3000.0,723.01,2385.54,723.03,1166.68,1321.64
2,task3,50.0,1.75,14.21,720.40,182.74,19.36
3,task4,200.0,27.41,43.63,605.64,324.00,33.57
4,task5,100.0,1.44,14.53,700.02,204.55,6.10
5,task6,500.0,82.69,185.80,502.73,354.26,175.95
6,Non_RT_Task,100.0,100.06,373.07,1254.75,1353.10,1373.84
